# Midterm Report 



## Background

Social media has become a new way of communication. It has revolutionized the relationship between users and digital products. The social media users are not only consumers, they are also content (comments) creators and spreaders. A massive amount of data has been generated directly by users and this allows us to measure people’s attention and attitude regarding a product in large scale. 

However, can the social media activities reflect our behaviors in real life?  Previous research showed that the tweets in a critical time period can successfully predict real world outcomes. A study on movie showed that both tweet - rate (number of tweets per hour) and sentiments could predict the box-office revenue (Asur, 2010) and the rating (Oghina,2012) of a particular movie. Also, the overall attitude of tweets has high correlations with people’s behavior in the stock market (Bollen,2011), political elections (Bermingham,2011), and etc.,

However, it may go beyond our expectations in the ways how these social media indicators are correlated with the real world outcomes. For example, products (De Vries,2012) and movies (e.g., [Tiny Times](https://en.wikipedia.org/wiki/Tiny_Times)) that have received negative comments were even more popular than those who received relatively higher ratings in social media. 
In this project, we are interested in how social media is related with users’ behavior in recreational activities. Also, we are curious about how the accuracy affected by factors such as the popularity of the social media and the diversity of information sources. 

Our hypothesis are:
- Both attention and attitude can predict the outcomes. Polarity will cause more attention 
- The popularity of social media and the diversity of information sources positively related with the accuracy.

In this project, we use movie as our subject area because:
- This subject area has research basis, which is both a good foundation of our project and a credible resource for comparing the project results.
- The real world outcomes (purchase behavior) can be easily measured by the box-office revenue.
- The “quality” of a product can be indicated by the IMDb score.




## Data-preparation & analysis

### 1. Movie Box Office Data  
   
   In the data preparation section, the first step of the project is to grasp movie data and twitter data separately:
For the movie data part, the main information we care about are movie’s name, total domestic box office amount (we narrow down the research area for only focus on North America region), movie’s release date, genre, and distributors. The first two features are the main identifier for the movie marked as our label. The third feature (release date) is used for targeting a specific time range for tweets search, and the rest of the features can be used for movie classification since we assume that correlations between twitter and a movie’s box office may also depends on movie’s type.

During the searching for box office data, we found that most of the movie box office information is either not complete or not freely available. There is no public free API available for large scale movie box office query. Therefore, we decide to write web scrapper by ourselves to collect box office data from [Boxoffice-Mojo](www.boxofficemojo.com). Boxoffice-Mojo is a website that tracks box office for more than 16,000 movies, basically, we use [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/) and python [urllib2](https://docs.python.org/2/library/urllib2.html) library  to do web data scrapping and saved the data into ‘movies_data.pkl’ file. 

In the following code, movie data is exported and sorted by its domestic total gross in decreasing order. The top 5 ranking movies are shown below:


In [1]:
import pickle
import pandas as pd
import datetime
import unirest
from glob import glob
from collections import Counter
from get_tweets import get_tweets

In [2]:
with open('movies_data.pkl', 'r') as picklefile:
    movies_scraped, movies_skipped = pickle.load(picklefile)

In [3]:
movies = pd.DataFrame(movies_scraped)
movies.dropna(axis=0, subset=['domestic_total_gross'], inplace=True)
movies.sort_values(by='domestic_total_gross', ascending=False, inplace=True)

In [4]:
print len(movies)
print movies[:5]

14682
              BOM_id                                             actors  \
9982       starwars7  John BoyegaDaisy RidleyAdam DriverOscar IsaacA...   
13275         avatar  Sam WorthingtonZoe SaldanaSigourney WeaverMich...   
1772   jurassicpark4  Nick RobinsonOmar SyChris PrattBryce Dallas Ho...   
2616      avengers11  Robert Downey, Jr.Chris HemsworthChris EvansJe...   
7211         titanic  Leonardo DiCaprioKate WinsletBilly ZaneKathy B...   

            budget         director  distributor  domestic_total_gross  \
9982   245000000.0      J.J. Abrams  Buena Vista           936662225.0   
13275          NaN    James Cameron          Fox           749766139.0   
1772   150000000.0  Colin Trevorrow    Universal           652270625.0   
2616   220000000.0      Joss Whedon  Buena Vista           623357910.0   
7211   200000000.0    James Cameron    Paramount           600788188.0   

                    genre                   movie_title  opening_income_wend  \
9982       Sci-Fi 

We can see that 14,682 movies’ information is dumped from the website and we checked our data by sorting their domestic total box office in descending order. We can see that the top 5 movies are:
* Star War VII        : 936,662,225 
* Avatar              : 749,766,139
* Jurassic Park IV    : 652,270,625
* Avengers II         : 623,357,910
* Titanic             : 600,788,188

It is already verified by other resource which means our data is reliable.
Here is the genre information for the collected movies and their counting distribution:

In [5]:
print movies['genre'].unique()
print movies['genre'].value_counts()

[u'Sci-Fi Fantasy' u'Sci-Fi Adventure' u'Action / Adventure' u'Romance'
 u'Animation' u'Action Thriller' u'Period Adventure' u'Sci-Fi Action'
 u'Fantasy' u'Historical Drama' u'Adventure' u'Action' u'Family Adventure'
 u'Sci-Fi Horror' u'Drama' u'Comedy / Drama' u'Horror' u'Family Comedy'
 u'Comedy' u'Sci-Fi Thriller' u'Horror Thriller' u'Sports Drama'
 u'Sci-Fi Comedy' u'Fantasy Comedy' u'Action Drama' u'Romantic Comedy'
 u'Action Comedy' u'Horror Comedy' u'Sci-Fi' u'Fantasy Drama' u'Thriller'
 u'War' u'Period Action' u'Action Horror' u'Historical Epic' u'Western'
 u'Crime Comedy' u'Adventure Comedy' u'Period Drama' u'Musical'
 u'Sports Comedy' u'Drama / Thriller' u'Crime Drama' u'Foreign / Action'
 u'Period Horror' u'Music Drama' u'Western Comedy' u'Documentary'
 u'War Drama' u'Sports Action' u'Period Comedy' u'Crime' u'Action / Crime'
 u'War Romance' u'IMAX' u'Action Fantasy' u'Crime Thriller'
 u'Romantic Thriller' u'Comedy Thriller' u'Family' u'Romantic Adventure'
 u'Concert' u'Fore

### 2. Twitter comments data
  
  When collecting twitter data, we also met some challenges. Our task is to collect tweets related to a specific film during the time period of the film release. By ‘related to a film’, we use keyword and hashtag search within one tweet. If one tweet contains a film’s name or hash-tag, we regard this tweet as related to this film. Unfortunately, Twitter [streaming API](https://dev.twitter.com/streaming/overview) only allows us to fetch the latest tweets within a week when given keywords and hashtag. Therefore, most of the films’ related tweets which posted earlier than a week cannot be fetched. We also tried several reliable open-sourced twitter data fetching library, since most of them are based on Twitter streaming API so they cannot satisfy our requirement. One of a not-very-popular library is found [here](https://github.com/Jefferson-Henrique/GetOldTweets-python). The basic idea is that when you enter on Twitter page a scroll loader starts, if you scroll down you start to get more and more tweets, all through calls to a JSON provider. After mimic we get the best advantage of Twitter Search on browsers, it can search the deepest oldest tweets. The good thing is that this library can fit most of our requirement, and the bad thing is that it is not a very reliable library due to its popularity, but so far we do not find any side effect or flaws when using this library.
 
Blocks below are codes we used to collect tweets data. Temporarily, we set the max tweets number for each movie to be 1,000. We will adjust this number in the future if the training/testing performance does not go well. Basic features we used for each tweet are its word count, retweet number, and whether it contains links. Besides, we conduct sentimental analysis from a [public API](http://text-processing.com/api/sentiment/). Due to the daily usage limitation for sentimental analysis API, we only choose seven films to do the sentimental analysis here, among the seven movies, four of them are popular movies which have very high box office,while the other threes are relatively normal movies:


In [16]:
time_range = datetime.timedelta(days=45)
maxtweets = 1000

def fetch_tweets(row):
    BOM_id = row['BOM_id']
    movie_title = row['movie_title']
    query = BOM_id + ' OR #' + BOM_id + ' OR ' + movie_title + ' OR #' + movie_title
    date = row['release_date']
    start = date.date().strftime('%Y-%m-%d')
    end = (date.date() + time_range).strftime('%Y-%m-%d')
    get_tweets(filename='data/' + BOM_id, maxtweets=maxtweets, query=query, since=start, until=end)
    print movie_title, start, end

In [18]:
fetched_movie = {}
directories = glob('data/*')
print  directories
for directory in directories:
    BOM_id = directory[5:]
    print BOM_id
    fetched_movie[BOM_id] = pd.read_csv(directory, sep=';', index_col=0)

['data/avatar', 'data/avengers11', 'data/jurassicpark4', 'data/misterlonely', 'data/starwars7', 'data/totalrecall2012rerelease', 'data/womenintrouble']
avatar
avengers11
jurassicpark4
misterlonely
starwars7
totalrecall2012rerelease
womenintrouble


In [19]:
def sentiment(text):
    # response = requests.post('http://text-processing.com/api/sentiment/', data={'text': text})
    response = unirest.post("https://japerk-text-processing.p.mashape.com/sentiment/",
        headers={
            "X-Mashape-Key": "EMrBfg9GO4mshgbevq2BtBZCdet3p1iXIWejsnKRDuWRNljYxI",
            "Content-Type": "application/x-www-form-urlencoded",
            "Accept": "application/json"
          },
        params={
            "language": "english",
            "text": text
          }
    )
    if response.code != 200:
        print response.code
        print response.body
        print 'failed: {}'.format(text)
        return {'sentiment': None, 'probability': None}
    sentiment = response.body['label']
    if sentiment == 'pos':
        return {'sentiment': 1, 'probability': response.body['probability']}
    elif sentiment == 'neg':
        return {'sentiment': -1, 'probability': response.body['probability']}
    else:
        return {'sentiment': 0, 'probability': response.body['probability']}

In [20]:
def get_sentiments(movie_dict):
    for BOM_id, movie in movie_dict.items():
        sentiments = pd.DataFrame([sentiment(text) for text in movie['text']])
        movie_dict[BOM_id] = pd.concat([movie, sentiments], axis=1)
        #movie.to_csv('data/' + BOM_id, sep=';')
        print BOM_id

In [21]:
def ouput_csv(movie_dict):
    for BOM_id, movie in movie_dict.items():
        movie.to_csv('data/' + BOM_id, sep=';')

From the following result we can see that, according to the sample we collected, popular movies tend to have more positive reviews and more tweets and retweets amount as a whole. More detailed research and their correlation will be studied after midterm report.

In [22]:
for title, movie in fetched_movie.items():
    counter = Counter(movie['sentiment'])
    print title, " : ", counter, 'total: ', sum(counter.values())

starwars7  :  Counter({0.0: 650, 1.0: 335, -1.0: 11, nan: 1, nan: 1}) total:  998
avengers11  :  Counter({1: 777, 0: 183, -1: 39}) total:  999
womenintrouble  :  Counter({0: 370, 1: 192, -1: 113}) total:  675
misterlonely  :  Counter({0: 16, 1: 16, -1: 6}) total:  38
totalrecall2012rerelease  :  Counter({-1: 635, 1: 253, 0: 111}) total:  999
avatar  :  Counter({1: 467, 0: 287, -1: 246}) total:  1000
jurassicpark4  :  Counter({0: 481, 1: 334, -1: 185}) total:  1000


## References
Asur, Sitaram, and Bernardo A. Huberman. "Predicting the future with social media." Web Intelligence and Intelligent Agent Technology (WI-IAT), 2010 IEEE/WIC/ACM International Conference on. Vol. 1. IEEE, 2010.

Bermingham, Adam, and Alan F. Smeaton. "On using Twitter to monitor political sentiment and predict election results." (2011).

Bollen, Johan, Huina Mao, and Xiaojun Zeng. "Twitter mood predicts the stock market." Journal of Computational Science 2.1 (2011): 1-8.

De Vries, Lisette, Sonja Gensler, and Peter SH Leeflang. "Popularity of brand posts on brand fan pages: An investigation of the effects of social media marketing." Journal of Interactive Marketing 26.2 (2012): 83-91.

Oghina, Andrei, et al. "Predicting imdb movie ratings using social media." European Conference on Information Retrieval. Springer Berlin Heidelberg, 2012.